# Quinta lista - Problema grupal 3

## Componentes del grupo

* Pol Casacuberta Gil
* Joaquin Faraone Prieto
* Agnès Felip i Díaz
* Marta Granero i Martí
3. Sumemos de nuevo
Nota: Este problema solo puede ser resuelto por grupos que resolvieron el problema de grupo
3 de la lista 3.
Los resultados del experimento de suma usando k-vecinos más cercanos fueron un poco decep-
cionantes incluso transformando los atributos para una mejor representación. Las relaciones no
lineales entre algunos atributos fueron las culpables de estos resultados no óptimos. Ahora que
conocemos otros clasificadores, podemos probar si pueden hacer un mejor trabajo.
El escenario será el mismo, aprendiendo a sumar números de tres dígitos con ejemplos definidos
por seis atributos, los tres primeros atributos corresponden a los tres dígitos del primer número
y los tres últimos a los dígitos del segundo número. La salida tiene cuatro valores, uno para cada
dígito para cada número del resultado de la suma. Por ejemplo:

a) Dado que vamos a utilizar clasificadores más potentes, probablemente necesitemos menos
ejemplos para funcionar mejor, así que generad dos conjuntos de datos de 25.000 y 50.000
ejemplos. Dividid los conjuntos de datos en un conjunto de test y entrenamiento con una
proporción de 90 %/10 %.

In [1]:
import pandas as pd
from pandas import read_csv

import numpy as np
from numpy.random import choice
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import set_config

from sklearn.metrics import  ConfusionMatrixDisplay,\
                  classification_report,  RocCurveDisplay, PrecisionRecallDisplay,\
                    accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error

from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.multioutput import MultiOutputClassifier


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB, CategoricalNB, MultinomialNB


from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV,train_test_split, cross_val_score, TimeSeriesSplit

from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.target.feature_correlation import feature_correlation
from yellowbrick.classifier import precision_recall_curve
from sklearn.preprocessing import KBinsDiscretizer


from apafib import load_electric_devices, load_energy

import warnings

set_config(display='text')
warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 16})
# sns.set()
pd.set_option('display.precision', 3)

In [3]:
def save_results(clf, X_test, y_test, nclf, df):
    df.loc[nclf,'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    df.loc[nclf,'precision score (W)'] = precision_score(y_test, clf.predict(X_test), average='weighted')
    df.loc[nclf,'recall score (W)'] = recall_score(y_test, clf.predict(X_test), average='weighted')
    df.loc[nclf,'f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='weighted')
    return df

results_df = pd.DataFrame()
niter = 15
cv = 5

data_columns = ['X1','X2','X3','X4','X5','X6','Y1','Y2','Y3','Y4']
X_columns = ['X1','X2','X3','X4','X5','X6']
y_columns = ['Y1','Y2','Y3','Y4']

#Function to create array
def prepareArray(number, size):
    X1 = list(map(int, str(number)))
    while len(X1) < size:
        X1.insert(0,0)
    return X1


def setFormat(first,second):
    total = first+second
    X1 = prepareArray(first,3)
    X2 = prepareArray(second,3)
    Y =  prepareArray(total,4)
    return X1+X2+Y

#setFormat(20,22) --Test
def createDataset(n):
    firsts= np.random.randint(0,1000, size=(n))
    seconds= np.random.randint(0,1000, size=(n))
    matrix = []

    for first, second in zip(firsts,seconds):
        matrix.append(setFormat(first,second))

    data = pd.DataFrame(matrix, columns=data_columns)
    data.drop_duplicates()

    return data

b) Como recordaréis, este es un problema de clasificación de salida múltiple, por lo que ne-
cesita usar el objeto MultiOutputClassifier de scikit learn. Esto envolverá los clasifi-
cadores para que pueda realizar este tipo de clasificación.
Usad SVC para la clasificación con un kernel rbf con diferentes valores para C. Podéis usar
diferentes potencias de 10 para ese parámetro para ver dónde está el mejor acierto para los
datos de test en lugar de usar validación cruzada, os tardará mucho si la usáis. Aseguraos
de usar el parámetro n_jobs de los objetos de clasificación de salida múltiple para que las
cosas se hagan en paralelo.


c) Usad GradientBoostingClassifier con diferentes valores para el número de estima-
dores y la profundidad máxima de los árboles. La cantidad de estimadores no tiene que
ser muy grande, pero probablemente necesitaréis alrededor de 50 árboles con la tasa de
aprendizaje predeterminada. Además, los árboles demasiado poco profundos darán malos
resultados. Evaluad el acierto sobre el test como en el apartado anterior.
Podéis acceder a los clasificadores individuales del clasificador de salida múltiple con el
atributo estimator_. Obtened la importancia de los atributos para cada clasificador que
encontraréis en el atributo feature_importances e interpretad los resultados.

d) ¿Por qué creéis que los resultados de estos clasificadores son mejores que los del clasificador k vecinos más cercanos? ¿Por qué creéis que necesitamos menos ejemplos para estos resultados?